# Ordered microstate analysis

In [18]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

from helper import *
from filenames_and_paths import *
from typing import *

In [19]:
mhw_objects: List[MicrostateHelperWrapper] = []
for i, target_filenames in enumerate(filenames):
    folders.end_folder = paths[i]
    # folders.end_folder = path045
    for filename in target_filenames:
        mhw_objects.append(MicrostateHelperWrapper.static_load(folders, filename + "_th"))

In [20]:
normative_maps = pd.read_csv(folders.save_data + "global_maps.csv", index_col=0)

In [21]:
t_normative_maps = normative_maps.T

In [22]:
# for i in range(4):
#     remap = {
#         0: "A",
#         1: "B",
#         2: "C",
#         3: "D"
#     }
#     mne.viz.plot_topomap(t_normative_maps[remap[i]], mhw_objects[0].raw.info)

In [23]:
for i, mhw in enumerate(mhw_objects):
    print(i, "-------------\n", mhw.raw_filename)
    (mhw
     .calc_normative_labels(t_normative_maps)
     .apply_normative_labels())

In [24]:
for mhw in mhw_objects:
    fig, axes = plt.subplots(2, 4, figsize=(10, 5))
    fig.suptitle(mhw.raw_filename)
    axes[0][0].set_title("Original")
    axes[1][0].set_title("Ordered")
    for i in range(4):
        mne.viz.plot_topomap(mhw.ms["Microstates"][i], mhw.raw.info, show=False, axes=axes[0][i])
        mne.viz.plot_topomap(mhw.ms_ordered["Microstates"][i], mhw.raw.info, show=False, axes=axes[1][i])

In [40]:
static_statistics = pd.DataFrame()

for mhw in mhw_objects:
    df = nk.microstates_static(mhw.ms_ordered, sampling_rate=2048)
    df["filename"] = mhw.raw_filename
    df["subject"] = mhw.folders.end_folder
    rest_state = mhw.raw_filename.split("_")[2]
    rest_state = rest_state.removeprefix("REST").removeprefix("rs")
    if len(rest_state) == 2:
        rest_state = rest_state[1]
    df["rest_state"] = rest_state
    static_statistics = pd.concat([static_statistics, df])

In [53]:
static_statistics.set_index(["subject", "rest_state"])

In [56]:
(static_statistics
 .reset_index(drop=True)
 .drop(["subject", "filename"], axis=1)
 .groupby('rest_state')
 .mean())

In [57]:
for mhw in mhw_objects:
    mhw.save()